<a href="https://colab.research.google.com/github/JuliethLopez/chatbot_simpsons/blob/master/Pre_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# librerias
import pickle
import re, string
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

# 1.Clasificador
### Cargar modelo

In [3]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip
   creating: clasificador/
   creating: clasificador/assets/
   creating: clasificador/variables/
  inflating: clasificador/variables/variables.data-00000-of-00002  
  inflating: clasificador/variables/variables.index  
  inflating: clasificador/variables/variables.data-00001-of-00002  
  inflating: clasificador/saved_model.pb  


In [4]:
modelo_clasificador = tf.keras.models.load_model('clasificador')
#Check its architecture
#modelo_clasificador.summary()

### Cargar para preprocesamiento
Cargamos el label_tokenizer, el encodeer, los trigramas y bigramas.

In [5]:
#label tokenizer
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/label_tokenizer.pickle', 'rb') as f:
  label_tokenizer = pickle.load(f)

In [6]:
#encoder
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/encoder.pickle', 'rb') as f:
  encoder = pickle.load(f)

In [7]:
#trigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqnew.pickle', 'rb') as f:
  freqnew = pickle.load(f)

In [8]:
#bigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqbnew.pickle', 'rb') as f:
  freqbnew = pickle.load(f)

### Función clasificadora

In [9]:
# funcion de frase tokenizada a palabras
reverse_word_map = dict(map(reversed, label_tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    # Busca palabras en el diccionario
    label = [reverse_word_map.get(i) for i in list_of_indices]
    return(label)

In [10]:
def clasifica (sentence):
  sentence = re.sub(' +', ' ',sentence.lower()) #sentence en minusculas
  sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence) #quita signos de puntuación
  # preprocesamiento trigramas
  vectorizer3 = CountVectorizer(vocabulary=freqnew.keys(), ngram_range=(3,3))
  X3 = vectorizer3.fit_transform([sentence])
  # preprocesamiento bigramas
  vectorizer2 = CountVectorizer(vocabulary=freqbnew.keys(), ngram_range=(2,2))
  X2 = vectorizer2.fit_transform([sentence])
  # concatenamos
  sequences_bt = np.concatenate((X3.toarray(), X2.toarray()), axis=1)
  # predicción
  pred = np.argmax(modelo_clasificador.predict(sequences_bt), axis=-1) #prediccion del modelo
  pred = encoder.inverse_transform(pred) #de label encoder a tokenizer
  pred = sequence_to_text(pred) #de tokenizer a label
  
  return (pred)

In [26]:
#ejemplo
sentence = "Would this be a good time to be honest?"
clasifica(sentence)

['homer_simpson']

# 2.Generador
### Cargar modelo

In [12]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/generador.zip
   creating: generador/
  inflating: generador/saved_model.pb  
   creating: generador/variables/
  inflating: generador/variables/variables.data-00000-of-00002  
  inflating: generador/variables/variables.index  
  inflating: generador/variables/variables.data-00001-of-00002  
   creating: generador/assets/


In [19]:
modelo_generador = tf.keras.models.load_model('generador')
#Check its architecture
#modelo_clasificador.summary()

### Cargar para preprocesamiento

In [20]:
# char a id
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/charxid.pickle', 'rb') as f:
  charxid = pickle.load(f)

In [21]:
#id a char
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/idxchar.pickle', 'rb') as f:
  idxchar = pickle.load(f)

### Función generadora

In [22]:
#funcion para generar texto usando el modelo pre-entrenado
def generate_text(start_string, num_generate):

  #convertimos el texto inicial en numeros (vectorizacion)
  input_eval = [charxid[s] for s in start_string] #vector columna
  input_eval = tf.expand_dims(input_eval, 0) #vector fila, agrega una diension

  #guardamos el texto predicho
  text_generated = []

  temperature = 1.0

  #tamaño del bache== 1
  modelo_generador.reset_states() #reinia o borra el estado recurrente de la red. Dejando valores aleatorios o ceros.
  for i in range(num_generate):
      predictions = modelo_generador(input_eval)
      #removemos la dimensión del batch, quita una dimensión
      predictions = tf.squeeze(predictions, 0)

      #usamos la distribución categorica para predecir la palabra que retorna el modelo
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      #utilizamos la palabra predicha y el estado oculto anterior como entrada
      input_eval = tf.expand_dims([predicted_id], 0) #devuelve un tensor con una dimensión adicional en el eje de índice

      text_generated.append(idxchar[predicted_id])

  return (start_string + ''.join(text_generated))

In [44]:
#ejemplo
sentence = "Homer:Would this be a good time to be honest?"
generate_text(sentence, 1000)

"Homer:Would this be a good time to be honest?\r\nC. Montgomery Burns: Okayyn: What did you tell my first cant animals? That, you're fluther cake expleaationally-line!\r\nMarge Simpson: Ow!\r\nHomer Simpson: They're thousands for experienced for I'm the nerd of the old one: The clossbecoming!\r\nMarge Simpson: Marge, fine, you're faultive. It's a claum.\r\nHomer Simpson: I had wrong?\r\nMarge Simpson: (INTRIGUED) And signed Moe Sprinkfoller!\r\nBart Simpson: Can't ever come up in there, meet-up Arust Ralph Wait! Yeah!\r\nHomer Simpson: How did you like more?\r\nComic Book Guy: (CONFUSED) Oh, that's what chapsswere middle they weren't try no pretty. You right with after minthes saying party.\r\nBart Simpson: Quite marriage. Yeah, everybody.\r\nHomer Simpson: Well, I can let 'em jocks. I wish I am a seems?\r\nAnnouncer: Moe name to have to can tell there down there?\r\nGOOD DROATH: (READY) Oh hah here not, Angeant.\r\nEnnter: (STROMES OF PAUGHTER) Bart, get inside the crazy nuclele 2926,

#Funciones para unir redes

In [30]:
import random
import re 

In [31]:
# frases de respuesta si no logra clasificar
frases_respuesta = ["Mmm, donuts.",
                      "Whatever, I'll be at Moe's.",
                      "Eat my shorts!",
                      "If anyone wants me, I'll be in my room.",
                      "Bye!",
                      "To start press any key. Where's the ANY key?",
                      "The lesson is, never try",
                      "I'm normally not a praying man, but if you're up there, please save me, Superman",
                      "You'll have to speak up I'm wearing a towel",
                      "I am so smart, S-M-R-T",
                    
                    "You're making a scene",
                      "Can I have some money now?"]

In [51]:
def union(lst1, lst2): 
    if lst1 != "<OOV>":
        final_list = lst1 + ": " + lst2 + "\n"
        return final_list
    else:
        a = random.randint(0, len(frases_respuesta)) #elige una frase aleatoriamente
        return frases_respuesta[a]

In [69]:
def output_ultimate(line):
  if line.split('\n')[1] == ' ':
    line2=line.split('\n')[2]
    labelsLine2 = line2[:line2.index(':')+1]
    line_ultimate = line2.replace(labelsLine2,"")
    print(line_ultimate)
  else:
    line2=line.split('\n')[1]
    labelsLine2 = line2[:line2.index(':')+1]
    line_ultimate = line2.replace(labelsLine2,"")
    print(line_ultimate)

# Pre chatbot

In [121]:
def prechatbot(Sentence):
  Pre_Etiqueta=clasifica(Sentence)
  display_one = ''.join(Pre_Etiqueta)
  Etiqueta = display_one[:display_one.index('_')].capitalize()
  Transicion = union(Etiqueta, Sentence).split('\n')[0]
  Linea = generate_text(Transicion, 1000)
  output_ultimate(Linea)

In [122]:
Usuario = "Would this be a good time to be honest?"
prechatbot(Usuario)

 Pite Door, whick you are the RV down hurt.
